# Neural Networks

NN은 torch.nn 패키지를 사용하여 생성할 수 있다.

nn은 모델을 정의하고 미분하는데 autograd를 사용한다. nn.Module은 layer와 output을 반환하는 forward(input) 메서드를 포함하고 있다.

숫자 이미지를 분류하는 신경망을 예제로 사용한다.

![](3.png)

위의 사진은 간단한 feed-forward network이다. input을 받아 여러 계층에 차례로 전달 후, output을 제공한다.

신경망의 일반적인 학습 과정  

1. 학습 가능한 매개변수를 갖는 신경망을 정의.
2. dataset 입력을 반복
3. 입력을 신경망에서 전파
4. loss를 계산
5. gradient를 신경망의 매개변수들에 역으로 전파
6. 신경망의 가중치를 갱신 (new_weight = weight - LR*gradient)

# 신경망 정의하기

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # input image channel = 1
        # output channel = 6
        # 5*5 convolutional matrix
        
        # Convolution kernel 정의
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        # Affine operation : y =Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # (2, 2) 크기 window에 대해 max_pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # 크기가 제곱수라면, 하나의 숫자만을 specify(특정)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # 배치 차원을 제외한 모든 차원을 하나로 flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면 gradient를 계산하는 backward 함수는 autograd를 사용하여 자동으로 정의된다. forward 함수에서는 어떠한 tensor 연산을 사용해도 된다.

모델의 학습 가능한 매개변수들은 net.parameters()에 의해 반환된다.

In [15]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 weight

10
torch.Size([6, 1, 5, 5])


임의의 32*32 입력을 넣어보자.

MNIST를 쓰기 위해서는 dataset의 image size를 32*32로 바꿔야 한다.

In [16]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 7.4480e-02, -1.4264e-01,  1.7299e-02,  1.1021e-01,  5.6953e-02,
         -1.3981e-05, -5.6526e-02, -2.4025e-02,  4.1463e-02, -6.5780e-02]],
       grad_fn=<AddmmBackward0>)


모든 매개변수의 gradient buffer를 0으로 설정하고, 무작위 값으로 back-propagation.

In [17]:
net.zero_grad()
out.backward(torch.randn(1,10))

torch.nn은 mini-batch만 지원한다.  
torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받는다.

예를 들어, nnConv2D는 nSamples* nChannels* height*width의 4차원 tensor를 입력으로 한다.  
만약 하나의 샘플만 있다면, input.unsqueeze(0)을 사용해서 가상의 차원을 추가한다.

# 요약

1. torch.Tensor : backward() 같은 autograd 연산을 지원하는 다차원 배열
2. nn.Module : 신경망 모듈. 매개변수를 캡슐화하는 간편한 방법
3. nn.Parameter : Tensor의 한 종류, Module에 속성으로 할당될 때 자동으로 매개변수로 등록
4. autograd.Function : autograd 연산의 순방향과 역방향 정의를 구현.

*지금까지 다룬 내용*
- 신경망 정의
- 입력을 처리하고 backward 호출하는 것

# Loss Function

loss function은 (output, target)을 한 쌍으로 입력 받아, output이 target으로부터 얼마나 멀리 떨어져 있는지를 추정하는 값을 계산.

nn 패키지에는 여러 loss function이 존재. 간단한 loss function으로는 출력과 대상간의 mean-squared error를 계산하는 nn.MSEloss가 있다.

In [18]:
output = net(input)
target = torch.randn(10) #예시를 위한 임의의 정답
target = target.view(1, -1) # 출력과 같은 shape으로 만듦
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.1723, grad_fn=<MseLossBackward0>)


이제 .grad_fn 속성을 이용해 loss를 역방향에서 따라가다보면, 이러한 모습의 연산 그래프를 볼 수 있다.

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> flatten -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss

따라서, loss.backward()를 실행할 때, 전체 그래프는 신경망의 매개변수에 의해 미분되며, 그래프 내의 requires_grad=True 인 모든 Tensor는 gradient가 누적된 .grad tensor를 갖게 된다.

In [19]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

# Backprop

error를 역전파하기 위해서는 loss.backward()만 해주면 된다. 기존에 계산된 gradient 값을 누적 시키고 싶지 않다면 기존에 계산된 gradient를 0으로 만드는 작업이 필요하다.

loss.backward()를 호출하여 역전파 전과 후에 conv1의 bias 변수의 gradient를 살펴보자.

In [20]:
net.zero_grad() # 모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0183,  0.0040,  0.0171, -0.0044, -0.0229,  0.0055])


# 가중치 갱신

실제로 많이 사용되는 가장 단순한 갱신 규칙은 Stochastic Gradient Descent 이다.

새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

In [21]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

신경망을 구성할 때, SGD, Adam, RMSProp 등과 같은 다양한 optimizer를 사용하고 싶을 수 있다.

이를 위해서 torch.optim 라는 작은 패키지에 이것들을 모두 구현해 두었다.

In [22]:
import torch.optim as optim

# optimizer를 생성한다.
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# 학습과정은 다음과 같다.
optimizer.zero_grad() #누적된 gradient 초기화
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # 업데이트 진행